In [1]:
import pandas as pd
import numpy as np
from subprocess import call, Popen
import subprocess
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

In [8]:
bin_num_limit = 8
stop_limit = 0.05
count_distr_limit = 0.05
INPUT_DIR = 'input'
Rscript_path = r'E:\App\R-3.5.1\bin\Rscript.exe'
woe_cols = [
    'FIELD_1', 'FIELD_10', 'FIELD_11', 'FIELD_12', 'FIELD_13', 'FIELD_14',
    'FIELD_15', 'FIELD_16', 'FIELD_17', 'FIELD_18', 'FIELD_19', 'FIELD_2',
    'FIELD_20', 'FIELD_21', 'FIELD_22', 'FIELD_23', 'FIELD_24', 'FIELD_25',
    'FIELD_26', 'FIELD_27', 'FIELD_28', 'FIELD_29', 'FIELD_3', 'FIELD_30',
    'FIELD_31', 'FIELD_33', 'FIELD_34', 'FIELD_35', 'FIELD_36', 'FIELD_37', 
    'FIELD_38', 'FIELD_39', 'FIELD_4', 'FIELD_40', 'FIELD_41', 'FIELD_42', 
    'FIELD_43', 'FIELD_44', 'FIELD_45', 'FIELD_46', 'FIELD_47', 'FIELD_48', 
    'FIELD_49', 'FIELD_5', 'FIELD_50', 'FIELD_51', 'FIELD_52', 'FIELD_53', 
    'FIELD_54', 'FIELD_55', 'FIELD_56', 'FIELD_57', 'FIELD_6', 
    'FIELD_8', 'FIELD_9', 'age_source1', 'age_source2'
]
commands = []
for col_name in woe_cols:
    os.makedirs(os.path.join(INPUT_DIR, "woe/%s"%col_name), exist_ok=True)
    df1 = train[["id", "label", col_name]]
    df2 = test[["id", col_name]]
    df1.to_csv(os.path.join(INPUT_DIR, "woe/%s/train.csv"%col_name), index=False, encoding="utf-8")
    df2.to_csv(os.path.join(INPUT_DIR, "woe/%s/test.csv"%col_name), index=False, encoding="utf-8")

    print("Binning: {0} bin_num_limit={1} stop_limit={2} count_distr_limit={3}".format(
    col_name, bin_num_limit, stop_limit, count_distr_limit))
    commands.append(
        "{0} woe.r {1}/woe {2} {3} {4} {5}".format(
        Rscript_path, INPUT_DIR, col_name, bin_num_limit, stop_limit, count_distr_limit)
    )

procs = [subprocess.Popen(c.strip().split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True) for c in commands]
for p in procs:
    (output, err) = p.communicate()
#     print(output, err)
#     print(p.wait())
# for c in commands:
#     os.system(c)
train_woe_df = {}
test_woe_df = {}
for col_name in woe_cols:
    df1 = pd.read_csv(os.path.join(INPUT_DIR, 'woe/%s/train_woe.csv'%col_name))
    df2 = pd.read_csv(os.path.join(INPUT_DIR, 'woe/%s/test_woe.csv'%col_name))
    for c in df1.columns:
        train_woe_df[c] = df1[c]
    for c in df2.columns:
        test_woe_df[c] = df2[c]

train_fe_df = pd.DataFrame.from_dict(train_woe_df)
test_fe_df = pd.DataFrame.from_dict(test_woe_df)

print(" "*4, "After dropping: ", train_fe_df.shape, test_fe_df.shape)
print("-"*40)

print("Done!")
print("-"*60)

Binning: FIELD_1 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: FIELD_10 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: FIELD_11 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: FIELD_12 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: FIELD_13 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: FIELD_14 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: FIELD_15 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: FIELD_16 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: FIELD_17 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: FIELD_18 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: FIELD_19 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: FIELD_2 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: FIELD_20 bin_num_limit=8 stop_limit=0.05 count_distr_limit=0.05
Binning: FIELD_21 bin_num_limit=8 stop_limit=0.05 cou

In [9]:
train_fe_df.to_csv(os.path.join(INPUT_DIR, "train_fe.csv"), index=False, encoding="utf-8")
test_fe_df.to_csv(os.path.join(INPUT_DIR, "test_fe.csv"), index=False, encoding="utf-8")